In [3]:
from sqlalchemy import create_engine, MetaData, Table
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session, sessionmaker
from sqlalchemy import Column, Integer, String, BigInteger, VARCHAR, SmallInteger
from sqlalchemy import *
import requests
from requests.auth import HTTPBasicAuth
from app_config import (
DIALECT,
DRIVER,
PORT,
HOST,
USER,
PASSWORD,
DATABASE
)

def to_dict(self):
    return {
        column.name: getattr(self, column.name)
        if not isinstance(getattr(self, column.name), datetime.datetime)
        else getattr(self, column.name).isoformat()
        for column in self.__table__.columns
    }

from models import Products

SQALCHEMY_DATABASE_URL = f"{DIALECT}+{DRIVER}://{USER}:{PASSWORD}@{HOST}/{DATABASE}"

engine = create_engine(
    SQALCHEMY_DATABASE_URL, 
    pool_recycle=3600,
    pool_pre_ping=True
)
Base = automap_base()
Base.prepare(engine, reflect=True)
Session = sessionmaker(bind=engine)
session = Session()


In [4]:
product_query = session.query(Products).all()


In [5]:
dict_list = [p.to_dict() for p in product_query]
#'product_id': 3, 'product_name': 'robust golden unsweetened oolong tea', 'price': 3.99, 'aisle_id': 94, 'department_id': 7}
#{'product_id': 4, 'product_name': 'smart ones classic favorites mini rigatoni with vodka cream sauce', 'price': 2.99, 'aisle_id': 38, 'department_id': 1}


#for p in dict_list:
#    print(p)
# """
# list(filter(lambda item: item['product_name'] == 'tea', dict_list))
#  note that you made need to use ifilter if the number of dictionaries is too high
# prints the folowing

# [{'product_id': 23826,
#   'product_name': 'tea',
#   'price': 3.99,
#   'aisle_id': 94,
#   'department_id': 7}]
# """
product_names = [p['product_name'] for p in dict_list]

In [ ]:
import pprint
##### for p in product_names:
#    api_call 
#r = requests.g
#url = "https://edamam-food-and-grocery-database.p.rapidapi.com/parser"

# url = "https://api.edamam.com/api/food-database/parser"
# products = {}

# querystring = {'ingr': product_names[11],
#                'app_id': '4ccefd3f',
#                'app_key': '5f95a55762f1cc4ed973d0c093d6ccad'   
#               }
# response = requests.get(url, params=querystring)

# pprint.pprint({ product_names[11] :
#     response.json()['hints'][0]['food']['nutrients']
#               })
url = "https://api.edamam.com/api/food-database/parser"
products = []

for x in product_names[:500]:
    try:
        querystring = {'ingr': x,
                       'app_id': '4ccefd3f',
                       'app_key': '5f95a55762f1cc4ed973d0c093d6ccad'   
                      }
        response = requests.get(url, params=querystring)

        products.append({ x :
                         response.json()['hints'][0]['food']['nutrients']
                      })
    except:
        pass
    
products

In [1]:
import pandas as pd

product_df = pd.DataFrame(products)

len(products)



NameError: name 'products' is not defined